In [1]:
# !unzip output.zip

In [2]:

!pip install spektral

     |████████████████████████████████| 122kB 6.5MB/s 


In [3]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [4]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb

In [29]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 1  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)

    features = sp.csr_matrix(x).astype(int)
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    labels = np.array(y).reshape((-1,1))

    print(adj.shape)
    print(features.shape)
    return adj, features, labels


class CircuitDataset(Dataset):
    def read(self):
        circuits = []
        circs = ['c6288', 'c17', 'c5315', 'c432', 'c499', 'c880', 'c1355', 'c1908', 'c3540']
        # circs = ['c17']
        for circ in circs:
            A, X, labels = load_data(circ, 'output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
            print(f"{circ}: {sum(labels)}, {len(labels)}")
        return circuits

dataset = CircuitDataset(transforms=NormalizeAdj())

# Parameters
F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
# [8 6 5 0 1 4 2] [3] [7]
# dataset_tr = dataset[idx_tr]
# dataset_va = dataset[idx_va]
# dataset_te = dataset[idx_te]
dataset_tr = dataset[[8,6,5,1,2, 0, 3, 4, 7]]
dataset_va = dataset[[0,3]]
dataset_te = dataset[4,7]
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)



(6288, 6288)
(6288, 4)
c6288: [846], 6288
(17, 17)
(17, 4)
c17: [2], 17
(5315, 5315)
(5315, 4)
c5315: [599], 5315
(432, 432)
(432, 4)
c432: [60], 432
(499, 499)
(499, 4)
c499: [50], 499
(880, 880)
(880, 4)
c880: [114], 880
(1355, 1355)
(1355, 4)
c1355: [192], 1355
(1908, 1908)
(1908, 4)
c1908: [257], 1908
(3540, 3540)
(3540, 4)
c3540: [406], 3540
[7 1 8 2 6 3 0] [4] [5]


In [30]:
N = sum([ g.n_nodes for g in dataset.graphs ])
n_out = 2

In [31]:
channels = 50          # Number of channels in the first layer
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping
# N = 

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((None, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GCNConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GCNConv(n_out,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./DetectFaultInCircuit',
)
callback_GCN = [tbCallBack_GCN]

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 4)            0           input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
gcn_conv_4 (GCNConv)            (None, 50)           200         dropout_4[0][0]                  
                                                                 input_6[0][0]              

In [32]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data import MixedLoader
from spektral.datasets.mnist import MNIST
from spektral.layers import GCNConv, GlobalSumPool
from spektral.layers.ops import sp_matrix_to_sp_tensor

In [33]:
optimizer = Adam()
loss_fn = SparseCategoricalCrossentropy()


# Training function
@tf.function
def train_on_batch(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs[:-1], training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
        acc = tf.reduce_mean(sparse_categorical_accuracy(target, predictions))

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc


# Evaluation function
def evaluate(loader):
    step = 0
    results = []
    for batch in loader:
        step += 1
        inputs, target = batch
        predictions = model(inputs[:-1], training=False)
        loss = loss_fn(target, predictions)
        acc = tf.reduce_mean(sparse_categorical_accuracy(target, predictions))
        results.append((loss, acc, len(target)))  # Keep track of batch size
        if step == loader.steps_per_epoch:
            results = np.array(results)
            return np.average(results[:, :-1], 0, weights=results[:, -1])

patience = 10
# Setup training
best_val_loss = 99999
current_patience = patience
step = 0

# Training loop
results_tr = []
for batch in loader_tr:
    step += 1

    # Training step
    inputs, target = batch
    loss, acc = train_on_batch(inputs, target)
    results_tr.append((loss, acc, len(target)))

    if step == loader_tr.steps_per_epoch:
        results_va = evaluate(loader_va)
        if results_va[0] < best_val_loss:
            best_val_loss = results_va[0]
            current_patience = patience
            results_te = evaluate(loader_te)
        else:
            current_patience -= 1
            if current_patience == 0:
                print("Early stopping")
                break

        # Print results
        results_tr = np.array(results_tr)
        results_tr = np.average(results_tr[:, :-1], 0, weights=results_tr[:, -1])
        print(
            "Train loss: {:.4f}, acc: {:.4f} | "
            "Valid loss: {:.4f}, acc: {:.4f} | "
            "Test loss: {:.4f}, acc: {:.4f}".format(
                *results_tr, *results_va, *results_te
            )
        )

        # Reset epoch
        results_tr = []
        step = 0

Train loss: 23.7684, acc: 0.2766 | Valid loss: 18.1345, acc: 0.1518 | Test loss: 5.5543, acc: 0.2368
Train loss: 15.3500, acc: 0.4334 | Valid loss: 1.6430, acc: 0.8113 | Test loss: 0.6522, acc: 0.7761
Train loss: 10.5897, acc: 0.6075 | Valid loss: 6.6873, acc: 0.8646 | Test loss: 0.6522, acc: 0.7761
Train loss: 9.5287, acc: 0.6908 | Valid loss: 9.4304, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 9.2426, acc: 0.7388 | Valid loss: 10.7475, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 8.8542, acc: 0.7503 | Valid loss: 10.4542, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 9.1948, acc: 0.7386 | Valid loss: 9.5996, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 8.7403, acc: 0.7358 | Valid loss: 8.4715, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 8.3533, acc: 0.7211 | Valid loss: 7.8803, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Train loss: 8.2028, acc: 0.7265 | Valid loss: 7.6514, acc: 0.8652 | Test loss: 0.6522, acc: 0.7761
Trai

In [17]:
data = dataset[4]

In [18]:
for batch in loader_tr:

Graph(n_nodes=499, n_node_features=4, n_edge_features=None, n_labels=1)

In [34]:
# Evaluate model
X_te = dataset[7].x
A_te = dataset[7].a
y_te = dataset[7].y

y_pred = model.predict([X_te, A_te], batch_size=X_te.shape[0])


In [36]:
np.argmax(y_pred,axis=1)

array([0, 0, 0, ..., 0, 0, 0])